In [14]:
import os, re, glob, numpy as np, torch, torch.nn as nn
from torchvision import transforms , models
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [19]:
import pandas as pd 
path = "/kaggle/input/prop-of-images/all_properties.csv"
df = pd.read_csv(path)

In [3]:
class JetImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        """
        images: numpy array (N, H, W, 3)
        labels: numpy array (N,) (not used for prediction)
        """
        self.images = images
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

In [4]:
def compute_physics_features(all_images, eps=1e-6):
    # all_images: (N, H, W, 3)
    ecal = all_images[:,:,:,0].astype(np.float32)
    hcal = all_images[:,:,:,1].astype(np.float32)
    tracks = all_images[:,:,:,2].astype(np.float32)
    mean_ratio = np.mean(ecal/(hcal+eps), axis=(1,2))
    mean_tracks = np.mean(tracks, axis=(1,2))
    mean_diff = np.mean(ecal-hcal, axis=(1,2))
    std_ecal = np.std(ecal, axis=(1,2))
    std_hcal = np.std(hcal, axis=(1,2))
    energy_asymmetry = (ecal - hcal)/(ecal + hcal + eps)
    mean_asymmetry = np.mean(energy_asymmetry, axis=(1,2))
    return mean_ratio, mean_tracks, mean_diff, std_ecal, std_hcal, mean_asymmetry

In [5]:
def compute_global_image_stats(chunk_paths, sample_fraction=0.1):
    sum_pixels = np.zeros(3, dtype=np.float64)
    sum_sq_pixels = np.zeros(3, dtype=np.float64)
    total_pixels = 0
    for chunk in chunk_paths:
        data = np.load(chunk)
        images = data['X_jets']
        N = images.shape[0]
        if sample_fraction < 1.0:
            sample_size = max(1, int(N*sample_fraction))
            idx = np.random.choice(N, sample_size, replace=False)
            images = images[idx]
        pixels = images.reshape(-1, 3).astype(np.float64)
        sum_pixels += pixels.sum(axis=0)
        sum_sq_pixels += (pixels**2).sum(axis=0)
        total_pixels += pixels.shape[0]
        del data, images, pixels
    mean = sum_pixels/total_pixels
    std = np.sqrt(sum_sq_pixels/total_pixels - mean**2)
    return mean.tolist(), std.tolist()

In [6]:
def load_model(model_class, checkpoint_dir, device):
    model = model_class(num_classes=2).to(device)
    pattern = os.path.join(checkpoint_dir, f"{model.__class__.__name__}_epoch_*.pth")
    files = glob.glob(pattern)
    if files:
        latest = max(files, key=lambda f: int(re.search(r'epoch_(\d+)', f).group(1)))
        ckpt = torch.load(latest, map_location=device)
        model.load_state_dict(ckpt['model_state'])
        print(f"Loaded {model.__class__.__name__} from {latest}")
    else:
        print(f"No checkpoint for {model.__class__.__name__}")
    model.eval()
    return model

In [25]:
def compute_physics_features_tensor(x, eps=1e-6):
    # x: (B, 3, H, W)
    ecal = x[:, 0:1, :, :]
    hcal = x[:, 1:2, :, :]
    tracks = x[:, 2:3, :, :]
    ratio = torch.mean(ecal / (hcal + eps), dim=[2,3])
    mean_tracks = torch.mean(tracks, dim=[2,3])
    diff = torch.mean(ecal - hcal, dim=[2,3])
    return torch.cat([ratio, mean_tracks, diff], dim=1)  # (B, 3)

In [26]:
class ChannelWiseConvBranch(nn.Module):
    def __init__(self, in_channels=1, out_channels=8):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0),
                nn.ReLU(),
                nn.AdaptiveAvgPool2d((1,1))
            ),
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.AdaptiveAvgPool2d((1,1))
            ),
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2),
                nn.ReLU(),
                nn.AdaptiveAvgPool2d((1,1))
            )
        ])
    def forward(self, x):
        outs = [conv(x) for conv in self.convs]
        outs = [o.view(o.size(0), -1) for o in outs]
        return torch.cat(outs, dim=1)  # (B, out_channels*3)

In [27]:
class FeaturePyramidBranch(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0),
                nn.ReLU(),
                nn.AdaptiveAvgPool2d((1,1))
            ),
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.AdaptiveAvgPool2d((1,1))
            ),
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2),
                nn.ReLU(),
                nn.AdaptiveAvgPool2d((1,1))
            )
        ])
    def forward(self, x):
        outs = [conv(x) for conv in self.convs]
        outs = [o.view(o.size(0), -1) for o in outs]
        return torch.cat(outs, dim=1)  # (B, out_channels*3)


In [28]:
class ResNet18PhysicsModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()  # backbone features
        
        # Physics branch
        self.physics_fc = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU()
        )
        
        # Channel-wise branch: applied to each channel separately
        self.channel_branch = ChannelWiseConvBranch(in_channels=1, out_channels=8)
        
        # Joint convolution branch (feature pyramid on full 3-channel input)
        self.joint_conv = FeaturePyramidBranch(in_channels=3, out_channels=8)
        
        # Additional Feature Pyramid branch on raw image
        self.fpn_branch = FeaturePyramidBranch(in_channels=3, out_channels=8)
        
        # Fusion fully connected layer: sum dimensions:
        # backbone: in_features; physics: 16; channel-wise: 3 channels * (8*3 = 24 each) = 72; joint: 8*3=24; fpn: 8*3=24.
        fusion_dim = in_features + 16 + 72 + 24 + 24
        self.fusion_fc = nn.Sequential(
            nn.Linear(fusion_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        backbone_feat = self.backbone(x)  # (B, in_features)
        phys_out = self.physics_fc(compute_physics_features_tensor(x))  # (B, 16)
        
        # Apply channel branch on each channel separately
        channel_feats = []
        for i in range(3):
            channel = x[:, i:i+1, :, :]  # (B, 1, H, W)
            feat = self.channel_branch(channel)  # (B, 24)
            channel_feats.append(feat)
        channel_feats = torch.cat(channel_feats, dim=1)  # (B, 24*3 = 72)
        
        joint_feat = self.joint_conv(x)  # (B, 24)
        fpn_feat = self.fpn_branch(x)    # (B, 24)
        
        fused = torch.cat([backbone_feat, phys_out, channel_feats, joint_feat, fpn_feat], dim=1)
        logits = self.fusion_fc(fused)
        return backbone_feat, logits

In [29]:
class EfficientNetB0PhysicsModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.backbone = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Identity()
        self.physics_fc = nn.Sequential(nn.Linear(3,16), nn.ReLU(), nn.Linear(16,16), nn.ReLU())
        self.channel_branch = ChannelWiseConvBranch(in_channels=1, out_channels=8)
        self.joint_conv = FeaturePyramidBranch(in_channels=3, out_channels=8)
        self.fpn_branch = FeaturePyramidBranch(in_channels=3, out_channels=8)
        fusion_dim = in_features + 16 + (8*3*3) + (8*3) + (8*3)
        self.fusion_fc = nn.Sequential(nn.Linear(fusion_dim,512), nn.ReLU(), nn.Linear(512,num_classes))
    def forward(self, x):
        backbone_feat = self.backbone(x)
        phys_out = self.physics_fc(compute_physics_features_tensor(x))
        channel_feats = []
        for i in range(3):
            channel_feats.append(self.channel_branch(x[:, i:i+1, :, :]))
        channel_feats = torch.cat(channel_feats, dim=1)
        joint_feat = self.joint_conv(x)
        fpn_feat = self.fpn_branch(x)
        fused = torch.cat([backbone_feat, phys_out, channel_feats, joint_feat, fpn_feat], dim=1)
        logits = self.fusion_fc(fused)
        return backbone_feat, logits

In [30]:
class DenseNet121PhysicsModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.backbone = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()
        self.physics_fc = nn.Sequential(nn.Linear(3,16), nn.ReLU(), nn.Linear(16,16), nn.ReLU())
        self.channel_branch = ChannelWiseConvBranch(in_channels=1, out_channels=8)
        self.joint_conv = FeaturePyramidBranch(in_channels=3, out_channels=8)
        fusion_dim = in_features + 16 + (8*3*3) + (8*3)
        self.fusion_fc = nn.Sequential(nn.Linear(fusion_dim,512), nn.ReLU(), nn.Linear(512,num_classes))
    def forward(self, x):
        backbone_feat = self.backbone(x)
        phys_out = self.physics_fc(compute_physics_features_tensor(x))
        channel_feats = []
        for i in range(3):
            channel_feats.append(self.channel_branch(x[:, i:i+1, :, :]))
        channel_feats = torch.cat(channel_feats, dim=1)
        joint_feat = self.joint_conv(x)
        fused = torch.cat([backbone_feat, phys_out, channel_feats, joint_feat], dim=1)
        logits = self.fusion_fc(fused)
        return backbone_feat, logits

In [31]:
class ViTPhysicsModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.backbone = models.vision_transformer.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
        in_features = self.backbone.heads.head.in_features
        self.backbone.heads.head = nn.Identity()
        self.physics_fc = nn.Sequential(nn.Linear(3,16), nn.ReLU(), nn.Linear(16,16), nn.ReLU())
        self.fusion_fc = nn.Sequential(nn.Linear(in_features+16,512), nn.ReLU(), nn.Linear(512,num_classes))
    def forward(self, x):
        emb = self.backbone(x)
        phys = self.physics_fc(compute_physics_features_tensor(x))
        fused = torch.cat([emb, phys], dim=1)
        logits = self.fusion_fc(fused)
        return emb, logits

In [32]:
class SwinPhysicsModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.backbone = models.swin_b(weights=models.Swin_B_Weights.DEFAULT)
        in_features = self.backbone.head.in_features
        self.backbone.head = nn.Identity()
        self.physics_fc = nn.Sequential(nn.Linear(3,16), nn.ReLU(), nn.Linear(16,16), nn.ReLU())
        self.fusion_fc = nn.Sequential(nn.Linear(in_features+16,512), nn.ReLU(), nn.Linear(512,num_classes))
    def forward(self, x):
        emb = self.backbone(x)
        phys = self.physics_fc(compute_physics_features_tensor(x))
        fused = torch.cat([emb, phys], dim=1)
        logits = self.fusion_fc(fused)
        return emb, logits

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_dir = "/kaggle/input/genie-extracted-dataset"
# List and sort chunks
all_files = [f for f in os.listdir(data_dir) if f.endswith(".npz")]
sorted_files = sorted(all_files, key=lambda f: int(re.search(r'chunk_(\d+)_', f).group(1)))
chunk_paths = [os.path.join(data_dir, f) for f in sorted_files]

# Compute global stats for normalization
global_mean, global_std = compute_global_image_stats(chunk_paths, sample_fraction=0.1)
print("Global stats:", global_mean, global_std)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(mean=global_mean, std=global_std)
])

Global stats: [8.247295636400174e-05, 5.1025032629110256e-05, 3.078809522143624e-05] [0.05153032077528415, 0.0019479140118328368, 0.0004856166605710416]


In [33]:
resnet_model = load_model(ResNet18PhysicsModel, "ckpt_resnet18", device)
efficientnet_model = load_model(EfficientNetB0PhysicsModel, "ckpt_efficientnet", device)
densenet_model = load_model(DenseNet121PhysicsModel, "ckpt_densenet", device)
vit_model = load_model(ViTPhysicsModel, "ckpt_vit", device)
swin_model = load_model(SwinPhysicsModel, "ckpt_swin", device)
models_list = [resnet_model, efficientnet_model, densenet_model, vit_model, swin_model]

No checkpoint for ResNet18PhysicsModel
No checkpoint for EfficientNetB0PhysicsModel
No checkpoint for DenseNet121PhysicsModel
No checkpoint for ViTPhysicsModel
No checkpoint for SwinPhysicsModel


In [34]:
def compute_physics_features(all_images, eps=1e-6):
    tabular_data_path = "/kaggle/input/properties-of-images/all_properties.csv"
    df = pd.read_csv(tabular_data_path)
# Assume the ordering of images in npz files corresponds to the order of rows in df
    df['image_preds'] = final_predictions
    df['mean_ratio'] = mean_ratio
    df['mean_tracks'] = mean_tracks
    df['mean_diff'] = mean_diff
    df['std_ecal'] = std_ecal
    df['std_hcal'] = std_hcal
    df['mean_asymmetry'] = mean_asymmetry
    print("Enriched tabular data with image predictions and advanced physics features.")

# --- Train Multiple Tabular Models ---
X = df.drop(columns=['y'])
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
ensemble_predictions = []
for chunk in tqdm(chunk_paths, desc="Processing Chunks"):
    data = np.load(chunk)
    images = data['X_jets']  # (N, H, W, 3)
    labels = data['y']       # not used for prediction here
    dataset = JetImageDataset(images, labels, transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    chunk_preds = []  # list to hold soft predictions from each model for this chunk
    for model in models_list:
        model.eval()
        model_preds = []
        with torch.no_grad():
            for imgs, _ in tqdm(dataloader, desc=f"Predicting with {model.__class__.__name__}", leave=False):
                imgs = imgs.to(device, dtype=torch.float)
                _, logits = model(imgs)
                prob = torch.softmax(logits, dim=1)[:, 1]  # probability for class 1
                model_preds.append(prob.cpu().numpy())
        model_preds = np.concatenate(model_preds)
        chunk_preds.append(model_preds)
    # Soft ensemble: average predictions from all models for this chunk
    ensemble_chunk = np.mean(np.array(chunk_preds), axis=0)
    ensemble_predictions.append(ensemble_chunk)
    del data, images, labels, dataset, dataloader

Predicting with ResNet18PhysicsModel: 100%|██████████| 313/313 [00:20<00:00, 14.52it/s]
                                                                                       
Predicting with EfficientNetB0PhysicsModel: 100%|██████████| 313/313 [00:22<00:00, 13.64it/s]
                                                                                             
Predicting with DenseNet121PhysicsModel: 100%|█████████▉| 312/313 [00:30<00:00, 10.39it/s]
                                                                                          
Predicting with ViTPhysicsModel: 100%|██████████| 313/313 [01:07<00:00,  5.45it/s]
                                                                                  
Predicting with ResNet18PhysicsModel: 100%|█████████▉| 312/313 [00:19<00:00, 16.03it/s]
                                                                                       
Predicting with EfficientNetB0PhysicsModel: 100%|█████████▉| 312/313 [00:22<00:00, 14.12it/s]
                  

In [ ]:
final_predictions = np.concatenate(ensemble_predictions)
print("Final ensemble predictions computed.")

In [ ]:
all_images_list = [np.load(c)['X_jets'] for c in chunk_paths]
all_images = np.concatenate(all_images_list)
mean_ratio, mean_tracks, mean_diff, std_ecal, std_hcal, mean_asymmetry = compute_physics_features(all_images)
print("Computed advanced physics features.")

In [ ]:
tabular_data_path = "/kaggle/input/properties-of-images/all_properties.csv"
df = pd.read_csv(tabular_data_path)
df['image_preds'] = binary_preds
df['mean_ratio'] = mean_ratio
df['mean_tracks'] = mean_tracks
df['mean_diff'] = mean_diff
df['std_ecal'] = std_ecal
df['std_hcal'] = std_hcal
df['mean_asymmetry'] = mean_asymmetry
print("Enriched tabular data with ensemble predictions and physics features.")

In [ ]:
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)
lgb_probs = lgb_model.predict_proba(X_test)[:,1]

# Train XGBoost
xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_probs = xgb_model.predict_proba(X_test)[:,1]

# Train CatBoost
cat_model = CatBoostClassifier(iterations=100, learning_rate=0.1, verbose=0, random_state=42)
cat_model.fit(X_train, y_train)
cat_probs = cat_model.predict_proba(X_test)[:,1]

# Train Histogram-based Gradient Boosting
hist_model = HistGradientBoostingClassifier(random_state=42)
hist_model.fit(X_train, y_train)
hist_probs = hist_model.predict_proba(X_test)[:,1]

# Soft ensemble of tabular models
ensemble_tab_probs = np.mean(np.array([lgb_probs, xgb_probs, cat_probs, hist_probs]), axis=0)
ensemble_tab_pred = (ensemble_tab_probs > 0.5).astype(int)
final_accuracy = accuracy_score(y_test, ensemble_tab_pred)
print(f"Final Ensemble Tabular Model Accuracy: {final_accuracy:.4f}")